In [1]:
import glob
import pandas as pd
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import tensorflow as tf
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
ignore_stopwords = ['what','why','how','against','who','where','myself']
STOPWORDS = set(stopwords.words('english')) - set(ignore_stopwords)

In [3]:
data_files = glob.glob("data/*.csv")
print(len(data_files))

13


In [4]:
data_list = []
for file in data_files:
    df = pd.read_csv(file, index_col=None, header=0)
    data_list.append(df)
data = pd.concat(data_list, axis=0, ignore_index=True)

#shuffle data
data = shuffle(data)
data.reset_index(inplace=True, drop=True)
data.head()

,Question,Answer,Tag
0,What are the fees for filing an appeal?,No Fees.,information
1,Is the PWDVA civil or criminal nature?,"While the remedies are civil in nature, the pr...",domestic
2,What is sexual abuse,When anyone by one's conduct of sexual nature ...,domestic
3,What happens after conviction,After completing hearing the convict on point ...,criminal
4,What is the jurisdiction of courts?,While the writ jurisdiction of the Supreme Cou...,information


In [5]:
data.describe()

,Question,Answer,Tag
count,887,887,887
unique,887,882,13
top,What is substantive Evidence,Yes.,criminal
freq,1,3,242


In [6]:
print(data['Tag'].unique())
print(len(data['Tag'].unique()))

['information' 'domestic' 'criminal' 'civil' 'vehicle' 'consumer' 'family'
 'education' 'employment' 'property' 'juvenile' 'general' 'immigration']
13


In [9]:
valid_data = pd.read_csv('validation.csv', index_col=None, header=0)
valid_data.head()

,Question,Answer,Tag,Paraphrased
0,I need some legal advice,"Sure , that's why I'm here",general,I need some legitimate guidance
1,"Stolen property purchased innocently, What to do?",At the end of the trial the property may go to...,criminal,"Taken property bought guiltlessly, What to do?"
2,What is encumbrance on a property?,"An encumbrance is a right to, interest in, or ...",property,What is encumbrance on a property?
3,What is Post Arrest Procedure,As per section 51 CrPC the Police Officer maki...,criminal,What is Post Arrest Procedure
4,How is a discrimination claim processed?,Anyone who is discriminated against may approa...,employment,How is a segregation guarantee prepared?


In [29]:
#lowercase
#remove punctuation,stopwords
#tokenize
#lemmatize


In [30]:
# normalize case and remove punctuation and remove stopwords
data['Question'] = data['Question'].str.lower()
punct_table = str.maketrans('','',string.punctuation)
data['Question'] = data['Question'].apply(lambda x:x.translate(punct_table))
data['Question'] = data['Question'].apply(lambda x:' '.join(x for x in x.split() if not x in STOPWORDS))
data

,Question,Answer,Tag
0,preparation crime punishable,Mere preparation to commit an offense is punis...,criminal
1,what offencecrime,"The Indian Penal Code (IPC), 1860 defines offe...",criminal
2,what order 37 code civil procedure cpc,Order 37 under CPC relates to the procedure in...,civil
3,payment wages act apply,It applies to the payment of wages to persons ...,employment
4,registration marriage compulsory divorce hindu...,No need. Registration of marriage is not neces...,civil
...,...,...,...
883,what shelter home,"The State Government may recognize, reputed an...",juvenile
884,child receive help child welfare committee ses...,Yes. A child in need of care and protection ma...,juvenile
885,need send legal notice filing case recovery money,"Yes, you need to send a legal notice in India ...",civil
886,parent ask specific child,An adoptive parent is allowed to ask for a chi...,family


In [11]:
valid_data['Paraphrased'] = valid_data['Paraphrased'].str.lower()
punct_table = str.maketrans('','',string.punctuation)
valid_data['Paraphrased'] = valid_data['Paraphrased'].apply(lambda x:x.translate(punct_table))
valid_data['Paraphrased'] = valid_data['Paraphrased'].apply(lambda x:' '.join(x for x in x.split() if not x in STOPWORDS))
valid_data['Paraphrased']

0                    need legitimate guidance
1      taken property bought guiltlessly what
2                   what encumbrance property
3                  what post arrest procedure
4          how segregation guarantee prepared
                        ...                  
195     point government joined involved suit
196               would able assist something
197                 what due date pay support
198                      what abrogation bail
199                    who made involved suit
Name: Paraphrased, Length: 200, dtype: object

In [12]:
def getLemmatize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

data['Question'] = data['Question'].apply(lambda x:getLemmatize(x))
valid_data['Paraphrased'] = valid_data['Paraphrased'].apply(lambda x:getLemmatize(x))
data

,Question,Answer,Tag
0,What are the fee for filing an appeal ?,No Fees.,information
1,Is the PWDVA civil or criminal nature ?,"While the remedies are civil in nature, the pr...",domestic
2,What is sexual abuse,When anyone by one's conduct of sexual nature ...,domestic
3,What happens after conviction,After completing hearing the convict on point ...,criminal
4,What is the jurisdiction of court ?,While the writ jurisdiction of the Supreme Cou...,information
...,...,...,...
882,Can such order regarding PWDVA be appealed aga...,The right to appeal is a statutory right that ...,domestic
883,What is encumbrance on a property ?,"An encumbrance is a right to, interest in, or ...",property
884,What are the section in CrPC for Trial in a Wa...,Section 244 to 247 and 249 CrPC.,criminal
885,How to enter into plea bargaining,File an application in the court in which tria...,criminal


In [13]:
#fixed params
NUM_CLASSES = len(data['Tag'].unique())
EMBEDDING_DIMENSION = 64
VOCABULARY_SIZE = 2000
MAX_LENGTH = 50
OOV_TOK = '<OOV>'
TRUNCATE_TYPE = 'post'
PADDING_TYPE = 'post'

In [14]:
tokenizer = Tokenizer(num_words=VOCABULARY_SIZE, oov_token=OOV_TOK)
tokenizer.fit_on_texts(data['Question'])
train_sequences = tokenizer.texts_to_sequences(data['Question'])

In [18]:
valid_sequences = tokenizer.texts_to_sequences(valid_data['Paraphrased'])

In [15]:
word_index = tokenizer.word_index
print('Vocabulary size:', len(word_index))
dict(list(word_index.items())[0:10])

Vocabulary size: 1385


{'<OOV>': 1,
 'the': 2,
 'what': 3,
 'is': 4,
 'a': 5,
 'of': 6,
 'to': 7,
 'can': 8,
 'in': 9,
 'for': 10}

In [16]:
maxlen = [len(seq) for seq in train_sequences]
ind = np.argmax(np.array(maxlen))
print("Max length of sequence - {}".format(maxlen[ind]))
print("Index - {}".format(ind))


Max length of sequence - 52
Index - 290


In [19]:
train_pad = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)
valid_pad = pad_sequences(valid_sequences, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)

In [20]:
print(data['Question'][10])
print(train_pad[10])

Anything else I need to know related to court case ?
[686 445  14  95   7 329 266   7  18  15   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [22]:

#convert tag column to type category
data["Tag"] = data["Tag"].astype('category')
data["Tag_Code"] = data["Tag"].cat.codes
data


,Question,Answer,Tag,Tag_Code
0,What are the fee for filing an appeal ?,No Fees.,information,9
1,Is the PWDVA civil or criminal nature ?,"While the remedies are civil in nature, the pr...",domestic,3
2,What is sexual abuse,When anyone by one's conduct of sexual nature ...,domestic,3
3,What happens after conviction,After completing hearing the convict on point ...,criminal,2
4,What is the jurisdiction of court ?,While the writ jurisdiction of the Supreme Cou...,information,9
...,...,...,...,...
882,Can such order regarding PWDVA be appealed aga...,The right to appeal is a statutory right that ...,domestic,3
883,What is encumbrance on a property ?,"An encumbrance is a right to, interest in, or ...",property,11
884,What are the section in CrPC for Trial in a Wa...,Section 244 to 247 and 249 CrPC.,criminal,2
885,How to enter into plea bargaining,File an application in the court in which tria...,criminal,2


In [23]:
codes = dict( enumerate(data['Tag'].cat.categories ) )
print(codes)

{0: 'civil', 1: 'consumer', 2: 'criminal', 3: 'domestic', 4: 'education', 5: 'employment', 6: 'family', 7: 'general', 8: 'immigration', 9: 'information', 10: 'juvenile', 11: 'property', 12: 'vehicle'}


In [36]:
inv_codes = dict(zip(codes.values(), codes.keys()))
valid_data['Tag_Code'] = valid_data["Tag"]
valid_data['Tag_Code'] = valid_data['Tag_Code'].replace(inv_codes)
valid_data

,Question,Answer,Tag,Paraphrased,Tag_Code
0,I need some legal advice,"Sure , that's why I'm here",general,need legitimate guidance,7
1,"Stolen property purchased innocently, What to do?",At the end of the trial the property may go to...,criminal,taken property bought guiltlessly what,2
2,What is encumbrance on a property?,"An encumbrance is a right to, interest in, or ...",property,what encumbrance property,11
3,What is Post Arrest Procedure,As per section 51 CrPC the Police Officer maki...,criminal,what post arrest procedure,2
4,How is a discrimination claim processed?,Anyone who is discriminated against may approa...,employment,how segregation guarantee prepared,5
...,...,...,...,...,...
195,When Government to be joined as a party to suit,When a suit is instituted against a public off...,civil,point government joined involved suit,0
196,Can you help me with something?,Sure what is it?,general,would able assist something,7
197,What is the due date to pay maintenance,Within 30 days of the date of announcing order,family,what due date pay support,6
198,What is cancellation of bail,Grounds for cancellation under Section 437(5) ...,criminal,what abrogation bail,2


In [29]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, SpatialDropout1D

In [30]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIMENSION, input_length=MAX_LENGTH))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(EMBEDDING_DIMENSION, dropout=0.3)))
model.add(Dense(EMBEDDING_DIMENSION*2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(EMBEDDING_DIMENSION, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

In [31]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 64)            88704     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 50, 64)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [33]:
ACCURACY_THRESHOLD = 0.99
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') >= ACCURACY_THRESHOLD):   
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
            self.model.stop_training = True
callbacks = myCallback()

In [38]:
EPOCHS = 200
history = model.fit(train_pad, np.array(data['Tag_Code']), epochs=EPOCHS, validation_data=(valid_pad,np.array(valid_data['Tag_Code'])), callbacks=[callbacks], verbose=1)

Epoch 1/200
28/28 [==============================] - 2s 60ms/step - loss: 3.1780 - accuracy: 0.1928 - val_loss: 2.8256 - val_accuracy: 0.2400
Epoch 2/200
28/28 [==============================] - 1s 31ms/step - loss: 2.7893 - accuracy: 0.2390 - val_loss: 2.6352 - val_accuracy: 0.2400
Epoch 3/200
28/28 [==============================] - 1s 30ms/step - loss: 2.6206 - accuracy: 0.2469 - val_loss: 2.4955 - val_accuracy: 0.2400
Epoch 4/200
28/28 [==============================] - 1s 29ms/step - loss: 2.4362 - accuracy: 0.2570 - val_loss: 2.3402 - val_accuracy: 0.2400
Epoch 5/200
28/28 [==============================] - 1s 35ms/step - loss: 2.1430 - accuracy: 0.2852 - val_loss: 2.2107 - val_accuracy: 0.2450
Epoch 6/200
28/28 [==============================] - 1s 29ms/step - loss: 2.0065 - accuracy: 0.2931 - val_loss: 2.1449 - val_accuracy: 0.2550
Epoch 7/200
28/28 [==============================] - 1s 33ms/step - loss: 1.9003 - accuracy: 0.3393 - val_loss: 2.1151 - val_accuracy: 0.2900
Epoch 

28/28 [==============================] - 2s 70ms/step - loss: 0.5460 - accuracy: 0.8016 - val_loss: 3.1317 - val_accuracy: 0.4550
Epoch 59/200
28/28 [==============================] - 2s 68ms/step - loss: 0.5491 - accuracy: 0.8083 - val_loss: 3.1665 - val_accuracy: 0.4350
Epoch 60/200
28/28 [==============================] - 2s 72ms/step - loss: 0.5252 - accuracy: 0.8151 - val_loss: 3.2636 - val_accuracy: 0.4250
Epoch 61/200
28/28 [==============================] - 2s 73ms/step - loss: 0.5181 - accuracy: 0.8129 - val_loss: 3.3746 - val_accuracy: 0.4550
Epoch 62/200
28/28 [==============================] - 2s 69ms/step - loss: 0.4901 - accuracy: 0.8230 - val_loss: 3.3482 - val_accuracy: 0.4450
Epoch 63/200
28/28 [==============================] - 2s 73ms/step - loss: 0.5069 - accuracy: 0.8331 - val_loss: 3.3959 - val_accuracy: 0.4450
Epoch 64/200
28/28 [==============================] - 2s 70ms/step - loss: 0.4993 - accuracy: 0.8196 - val_loss: 3.1980 - val_accuracy: 0.4700
Epoch 65/200

Epoch 115/200
28/28 [==============================] - 2s 73ms/step - loss: 0.2238 - accuracy: 0.9549 - val_loss: 4.0767 - val_accuracy: 0.4950
Epoch 116/200
28/28 [==============================] - 2s 65ms/step - loss: 0.1790 - accuracy: 0.9605 - val_loss: 3.9709 - val_accuracy: 0.5300
Epoch 117/200
28/28 [==============================] - 2s 66ms/step - loss: 0.2039 - accuracy: 0.9538 - val_loss: 4.2585 - val_accuracy: 0.5000
Epoch 118/200
28/28 [==============================] - 2s 71ms/step - loss: 0.2057 - accuracy: 0.9605 - val_loss: 4.1215 - val_accuracy: 0.5300
Epoch 119/200
28/28 [==============================] - 2s 70ms/step - loss: 0.1902 - accuracy: 0.9617 - val_loss: 3.9794 - val_accuracy: 0.4800
Epoch 120/200
28/28 [==============================] - 2s 80ms/step - loss: 0.1826 - accuracy: 0.9718 - val_loss: 4.0331 - val_accuracy: 0.4950
Epoch 121/200
28/28 [==============================] - 2s 83ms/step - loss: 0.1669 - accuracy: 0.9718 - val_loss: 4.3195 - val_accuracy:

28/28 [==============================] - 2s 59ms/step - loss: 0.0961 - accuracy: 0.9876 - val_loss: 4.2244 - val_accuracy: 0.5100
Epoch 172/200
28/28 [==============================] - 1s 47ms/step - loss: 0.0969 - accuracy: 0.9865 - val_loss: 4.3689 - val_accuracy: 0.5250
Epoch 173/200
28/28 [==============================] - 1s 51ms/step - loss: 0.1105 - accuracy: 0.9853 - val_loss: 4.5126 - val_accuracy: 0.5400
Epoch 174/200
28/28 [==============================] - 1s 52ms/step - loss: 0.1081 - accuracy: 0.9842 - val_loss: 4.7898 - val_accuracy: 0.5250
Epoch 175/200
28/28 [==============================] - 1s 51ms/step - loss: 0.1187 - accuracy: 0.9786 - val_loss: 4.2533 - val_accuracy: 0.5600
Epoch 176/200
28/28 [==============================] - 1s 49ms/step - loss: 0.1013 - accuracy: 0.9842 - val_loss: 4.5150 - val_accuracy: 0.5250
Epoch 177/200
27/28 [===========================>..] - ETA: 0s - loss: 0.0834 - accuracy: 0.9907
Reached 99.00% accuracy, so stopping training!!
28/28

In [45]:

import pickle
# save model
model.save('intent_model.h5',history)
# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
# save catgeory codes
import pickle
codes = dict( enumerate(data['Tag'].cat.categories ) )
with open('cat_codes.pickle', 'wb') as handle:
    pickle.dump(codes, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [46]:
message = ["how can i register property online"]
input_seq = tokenizer.texts_to_sequences(message)
padded_input = pad_sequences(input_seq,maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)
print(padded_input)
prediction_probs = model.predict(padded_input)
prediction = np.argmax(prediction_probs)

[[  3 224   1 432  12 357   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]]


In [47]:
codes = dict(enumerate(data['Tag'].cat.categories ))
print(codes)
print(codes[prediction])

{0: 'civil', 1: 'consumer', 2: 'criminal', 3: 'domestic', 4: 'education', 5: 'employment', 6: 'family', 7: 'general', 8: 'immigration', 9: 'information', 10: 'juvenile', 11: 'property', 12: 'vehicle'}
property
